In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pickle
import glob
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [2]:
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.2.0


In [3]:
# lstm autoencoder recreate sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard

In [4]:
RUN_NAME = '190719_Hela_Ecoli_1to1_01'
EXPERIMENT_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test'
ENCODED_FEATURES_DIR = '{}/encoded-features/{}'.format(EXPERIMENT_DIR, RUN_NAME)
VECTOR_SEQUENCES_DIR = '{}/vector-sequences'.format(ENCODED_FEATURES_DIR)
CUBOID_VECTOR_DIR = '{}/cuboid-vectors'.format(ENCODED_FEATURES_DIR)

In [5]:
# load all the vector sequences
vector_sequences_l = glob.glob("{}/vector-sequence-feature-*.pkl".format(VECTOR_SEQUENCES_DIR))
feature_vector_sequences_l = []
feature_ids_l = []
scaler = MinMaxScaler(feature_range=(0.0,1.0))
for vector_sequence in vector_sequences_l:
    feature_id = int(os.path.basename(vector_sequence).split('-')[3].split('.')[0])
    feature_ids_l.append(feature_id)
    with open(vector_sequence, 'rb') as f:
        feature_vectors_l = pickle.load(f)
        feature_vectors_arr = np.array(feature_vectors_l)

        scaler = scaler.fit(feature_vectors_arr)
        normalised = scaler.transform(feature_vectors_arr)

        feature_vector_sequences_l.append(normalised)


In [6]:
feature_vector_sequences_l[2].shape

(17, 3136)

In [7]:
feature_vector_sequences_l[2]

array([[0.        , 0.        , 0.0370321 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.99999994, 0.        , 0.13332427, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.7283912 , 0.61314243, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.8476    , 0.7408502 , ..., 0.        , 0.        ,
        0.        ],
       [0.61273605, 0.6172964 , 0.57815593, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [8]:
feature_vector_sequences_l = tf.keras.preprocessing.sequence.pad_sequences(feature_vector_sequences_l, padding='pre', value=0.0, dtype='float32')

In [9]:
feature_vector_sequences_l[2]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.7283912 , 0.61314243, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.8476    , 0.7408502 , ..., 0.        , 0.        ,
        0.        ],
       [0.61273605, 0.6172964 , 0.57815593, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [10]:
len(feature_vector_sequences_l)

199

In [11]:
# define input sequence
feature_vector_sequences = np.array(feature_vector_sequences_l)

In [12]:
feature_vector_sequences.min()

0.0

In [13]:
feature_vector_sequences[2]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.7283912 , 0.61314243, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.8476    , 0.7408502 , ..., 0.        , 0.        ,
        0.        ],
       [0.61273605, 0.6172964 , 0.57815593, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [14]:
feature_vector_sequences[20].shape

(165, 3136)

In [15]:
# reshape input into [samples, timesteps, features]
n_samples = len(feature_vector_sequences_l)
timesteps = feature_vector_sequences_l[1].shape[0]
n_features = feature_vector_sequences_l[1].shape[1]
feature_vector_sequences = feature_vector_sequences.reshape((n_samples, timesteps, n_features))

In [16]:
feature_vector_sequences.shape

(199, 165, 3136)

In [17]:
np.isnan(np.sum(feature_vector_sequences))

False

In [30]:
# define model
model = Sequential()

model.add(LSTM(units=1000, activation='relu', input_shape=(timesteps,n_features), return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(units=500, activation='relu', input_shape=(timesteps,n_features), return_sequences=False, recurrent_dropout=0.2))

model.add(RepeatVector(timesteps))

model.add(LSTM(units=500, activation='relu', return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(units=1000, activation='relu', return_sequences=True, recurrent_dropout=0.2))

model.add(TimeDistributed(Dense(n_features)))

In [20]:
optimizer = optimizers.Adam(learning_rate=0.0001, clipvalue=0.5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                405632    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 165, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 165, 32)           8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 165, 32)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, 165, 3136)         103488    
Total params: 517,440
Trainable params: 517,440
Non-trainable params: 0
________________________________________________

In [21]:
# split the data set into training and test
X_train, X_test, _, _ = train_test_split(feature_vector_sequences, feature_vector_sequences, test_size=0.10)

In [22]:
X_train.shape

(179, 165, 3136)

In [23]:
# divide X_test into validation and test
split_num = int(len(X_test)/2)
X_val = X_test[:split_num]
X_test = X_test[split_num:]

In [24]:
X_train.shape, X_test.shape, X_val.shape

((179, 165, 3136), (10, 165, 3136), (10, 165, 3136))

In [25]:
# set up TensorBoard
logdir = "/tmp/lstm/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [26]:
batch_size = len(X_train)
epochs = 500

In [27]:
# fit model
history = model.fit(X_train, X_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, X_val),
                    callbacks=[tensorboard_callback],
                    shuffle=True)

Epoch 1/500
1/1 [==============================] - 1s 613ms/step - loss: 0.0213 - val_loss: 0.0184
Epoch 2/500
1/1 [==============================] - 0s 97ms/step - loss: 0.0211 - val_loss: 0.0183
Epoch 3/500
1/1 [==============================] - 0s 95ms/step - loss: 0.0209 - val_loss: 0.0182
Epoch 4/500
1/1 [==============================] - 0s 94ms/step - loss: 0.0209 - val_loss: 0.0182
Epoch 5/500
1/1 [==============================] - 0s 98ms/step - loss: 0.0208 - val_loss: 0.0182
Epoch 6/500
1/1 [==============================] - 0s 91ms/step - loss: 0.0208 - val_loss: 0.0182
Epoch 7/500
1/1 [==============================] - 0s 97ms/step - loss: 0.0208 - val_loss: 0.0181
Epoch 8/500
1/1 [==============================] - 0s 98ms/step - loss: 0.0208 - val_loss: 0.0181
Epoch 9/500
1/1 [==============================] - 0s 103ms/step - loss: 0.0207 - val_loss: 0.0181
Epoch 10/500
1/1 [==============================] - 0s 94ms/step - loss: 0.0207 - val_loss: 0.0181
Epoch 11/500
1/1 

In [ ]:
# connect the encoder LSTM as the output layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)

In [48]:
# get the feature vector for the input sequence
yhat = model.predict(sequence)
print(yhat.shape)

(199, 10)


In [39]:
yhat[38]

array([6.4656757e-02, 4.2602994e-20, 6.0735204e-19, 1.7050762e-01,
       8.3085743e-20, 4.3444159e-20, 1.6282232e-19, 5.8913608e-19,
       5.8449286e-23, 2.1671874e-19], dtype=float32)

In [49]:
FEATURE_ID_LIST_FILE = '{}/feature_ids.pkl'.format(ENCODED_FEATURES_DIR)

In [52]:
features_df = pd.read_pickle(FEATURE_ID_LIST_FILE)

In [53]:
features_df.head()

,sequence,charge,feature_id
0,IVGYFVSGCDPSIMGIGPVPAISGALK,3,7392101
1,VVDLMAHMASKE,2,2698201
2,TDNAGDQHGGGGGGGGGAGAAGGGGGGENYDDPHK,4,325001
3,TDNAGDQHGGGGGGGGGAGAAGGGGGGENYDDPHK,4,325002
4,ITESEEVVSR,2,858201


In [54]:
len(features_df)

199

In [55]:
count_l = []
for group_name,group_df in features_df.groupby(['sequence','charge'], as_index=False):
    count_l.append((group_name[0], group_name[1], len(group_df)))
count_df = pd.DataFrame(count_l, columns=['sequence','charge','count'])

In [56]:
count_df

,sequence,charge,count
0,AALESTLAAITESLK,2,1
1,AAQSNENLSDSQQEPPK,2,1
2,ADPDVTLAGQEAIFILAR,2,3
3,ALANVNIGSLICNVGAGGPAPAAGAAPAGGPAPSTAAAPAEEK,3,3
4,ALYPGALGRPPPMPPMNFDPR,3,2
...,...,...,...
94,VVQYNINTEELYSYLK,2,2
95,YANEVNSDAGAFK,2,2
96,YEQGTGCWQGPNR,2,2
97,YPEETLSLMTK,2,4


In [58]:
sequence = 'YPEETLSLMTK'
charge = 2
features_df[(features_df.sequence == sequence) & (features_df.charge == charge)]

,sequence,charge,feature_id
70,YPEETLSLMTK,2,3639801
71,YPEETLSLMTK,2,3639802
72,YPEETLSLMTK,2,3639803
73,YPEETLSLMTK,2,3639804


In [62]:
feature_sequence_vector_l = []
for idx in range(len(feature_ids_l)):
    feature_sequence_vector_l.append((feature_ids_l[idx], yhat[idx]))

In [63]:
feature_sequence_vector_l[4]

(7066402,
 array([0.        , 0.        , 0.00341782, 0.        , 0.24347614,
        0.        , 0.14170818, 0.        , 0.        , 0.        ],
       dtype=float32))

In [66]:
feature_sequence_vector_df = pd.DataFrame(feature_sequence_vector_l, columns=['feature_id','cuboid_vector'])

In [67]:
encoded_features_df = pd.merge(feature_sequence_vector_df, features_df, how='inner', left_on=['feature_id'], right_on=['feature_id'])

In [70]:
encoded_features_df.sample(n=10)

,feature_id,cuboid_vector,sequence,charge
153,841602,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",ETEAIGHVVR,2
105,4488001,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",IIPGFMCQGGDFTR,2
29,2210901,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",SLGDDISSETSGDFRK,2
38,325001,"[1.137137e-20, 0.0, 0.0034178158, 0.0, 0.24347...",TDNAGDQHGGGGGGGGGAGAAGGGGGGENYDDPHK,4
65,1164702,"[1.7240611e-19, 5.121458e-20, 0.0034178158, 0....",YEQGTGCWQGPNR,2
89,7144202,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",FYPEDVSEELIQDITQR,3
183,200101,"[2.2999198e-17, 7.0955246e-24, 0.0034177017, 0...",KDDTDDEIAK,2
109,6904801,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",AALESTLAAITESLK,2
31,1251202,"[0.0, 0.0, 0.0034178176, 0.0, 0.24347614, 0.0,...",DAHLLVESK,2
151,2918002,"[0.0, 0.0, 0.0034178158, 2.7521634e-23, 0.2434...",QYDSVECPFCDEVSK,2


In [71]:
encoded_features_df[(encoded_features_df.sequence == sequence) & (encoded_features_df.charge == charge)]

,feature_id,cuboid_vector,sequence,charge
63,3639804,"[0.0, 0.0, 0.0034178176, 0.0, 0.24347614, 0.0,...",YPEETLSLMTK,2
77,3639803,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",YPEETLSLMTK,2
79,3639802,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",YPEETLSLMTK,2
93,3639801,"[0.0, 0.0, 0.0034178158, 0.0, 0.24347614, 0.0,...",YPEETLSLMTK,2


In [79]:
from scipy import spatial

In [79]:
feature_id = 3639801
reference = encoded_features_df[(encoded_features_df.feature_id == feature_id)].iloc[0].cuboid_vector


encoded_features_df['similarity'] = encoded_features_df.apply(lambda row: spatial.distance.cosine(row.cuboid_vector, reference), axis=1)

/Users/darylwilding-mcbride/anaconda3/envs/tf/lib/python3.8/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [95]:
from numpy import dot
from numpy.linalg import norm

a = reference
b = encoded_features_df[(encoded_features_df.feature_id == 325001)].iloc[0].cuboid_vector
cos_sim = dot(a, b)/(norm(a)*norm(b))

In [96]:
cos_sim

1.0